## Изучение общей информации

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

In [2]:
df = pd.read_csv('credit_score.csv')

In [3]:
df.head(10) #Вызываем первые 10 строк для ознакомления с содержанием

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Просмотр первых 10 строк уже показал, что у данных в столбце education разный регистр. Общий трудовой стаж указан с минусом, стаж и доход приведены в дробях, нужно привести к целым числам. Просмотрим общую информацию о столбцах и проверим на дубликаты.

In [4]:
df.duplicated().sum() #число дубликатов

54

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Таблица содержит 54 дубликата. Столбцы `days_employed` и `total_income` содержат в себе пропуски, причем одинаковое количество. Проверим, если пропуски находятся в одинаковых строчках.

In [6]:
df.days_employed.isna().sum() #Проверяем количество пустых ячеек в days_employed

2174

In [7]:
df.loc[df.days_employed.isna()] #Проверяем соответствуют ли пустые значения days_employed и total_income

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Пустые ячейки присутствуют у ~10% ячеек датафрейма, удалять значения нельзя.
Пустые ячейки days_employed соответствуют пустым ячейкам у total_income. 
Предполагаем, что данные о ежемесячном доходе отсутствуют, так как нет информации о стаже работы клиента.
Проверим остальные категориальные данные.

In [8]:
df.education.unique() 

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Как было указано выше, разные регистры в названиях

In [9]:
df.family_status.unique() 

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

"Не женат / не замужем" написано с большой буквы, в отличие от "женат / замужем"

In [10]:
df.gender.unique() 

array(['F', 'M', 'XNA'], dtype=object)

XNA может быть как ошибка при введении данных, так и третий гендер

In [11]:
df.income_type.unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [12]:
df.purpose.unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

В этом столбце много дупликатов по смыслу, что исказит выводы при анализе.
Проверим количественные данные с помощью `describe()`

In [13]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


20 детей это слишком много, а -1 это слишком мало. Скорее всего ошибка при вводе данных, имелось в виду 2 и 1 ребенок соответственно.
Разница между min и max у стажа и дохода существенна. Пропуски нужно заполнять медианой.
Кроме того, в столбце `dob_years` имеются значения, равные нулю. Подозреваем отсутствие данных по возрасту клиента

**Вывод**

1) Необходимо удалить дубликаты, а также обработать пропуски в столбцах `days_employed` и `total_income`. Единственное значение у столбца `gender`, равное `"XNA"`, допустимо удалить.  
2) Значения в `days_employed` нужно привести к положительным, а в `days_employed` и `total_income` к целым для лучшей визуализации.  
3) В `education` привести данные к одному регистру  
4) В `purpose` необходима лемматизация данных

## Предобработка данных

Ранее мы обнаружили странные значения в столбце `children`, а именно 20 и -1. Заменим их на 2 и 1 соответственно. Кроме того, в столбце `family_status` "Не женат / не замужем" написано с большой буквы.

In [14]:
df.children.replace({20: 2, -1: 1}, inplace = True) #Заменяем все значения 20 в столбце children на 2, аналогично для (-1)
df.children.value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [15]:
df.family_status.replace({'Не женат / не замужем': 'не женат / не замужем'}, inplace = True)
df.family_status.value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

### Обработка пропусков

При получении общей информации пропуски были обнаружены в столбцах `days_employed` и `total_income`. Поскольку они находятся в существенном (около 10%) количестве строк, простое удаление может исказить общую картину. Мы решили заменить пропуски на медианные значения, так как среднее значение по стажу и доходам скорее всего неточное. Перед этим удалим строчку с неизвестным гендером, чтобы она не повлияла на медианные значения. Так как значения в стаже у нас отрицательные, после заменим их на положительные.

In [16]:
df[df.gender == 'XNA'] #вызываем строку с XNA в гендере

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


In [17]:
df = df[df.gender != 'XNA']
df[df.gender == 'XNA'] #возвращает пустой датафрейм

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Заменяем значения в `days_employed` на положительные с помощью модуля

In [18]:
df.days_employed = df.days_employed.abs() #Берем модуль от каждого значения и проверяем столбец
df.head()                                                                         

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [19]:
df.days_employed = df.days_employed.fillna(df.days_employed.median()) #Заменяем NaN на медианное значение по столбцу
df.days_employed.isna().sum() #проверяем результат

0

In [20]:
df.total_income = df.total_income.fillna(df.days_employed.median()) #Повторяем для столбца с доходом
df.total_income.isna().sum() 

0

**Вывод**

Мы удалили артефакт из gender и заменили отсутствующие значения в `days_employed` и `income_type` на медианные, чтобы не потерять 10% данных. Перед этим значения стажа были приведены к положительным

### Замена типа данных

Заменим типы данных столбцов `days_employed` и `total_income` с дробных на целые для лучшей визуализации

In [21]:
df.days_employed = df.days_employed.astype('int')
df.total_income = df.total_income.astype('int')
df.info() #типы колонок стали int64

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21524 non-null  int64 
 1   days_employed     21524 non-null  int32 
 2   dob_years         21524 non-null  int64 
 3   education         21524 non-null  object
 4   education_id      21524 non-null  int64 
 5   family_status     21524 non-null  object
 6   family_status_id  21524 non-null  int64 
 7   gender            21524 non-null  object
 8   income_type       21524 non-null  object
 9   debt              21524 non-null  int64 
 10  total_income      21524 non-null  int32 
 11  purpose           21524 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 2.0+ MB


In [22]:
df.head() #теперь данные по стажу и доходам лучше читаются

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**

Мы изменили тип данных в столбцах `days_employed` и `total_income` на `int`. Теперь данные лучше воспринимаются и с ними легче работать.

### Обработка дубликатов

В начале работы мы обнаружили как полные дубликаты, так и неявные в столбце education. Сначала обработаем полные дубликаты

In [23]:
df.loc[df.duplicated(), :] #вызываем строчки, которые повторяются во всем датафрейме

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,2194,41,среднее,1,женат / замужем,0,F,сотрудник,0,2194,покупка жилья для семьи
4182,1,2194,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,2194,свадьба
4851,0,2194,60,среднее,1,гражданский брак,1,F,пенсионер,0,2194,свадьба
5557,0,2194,58,среднее,1,гражданский брак,1,F,пенсионер,0,2194,сыграть свадьбу
7808,0,2194,57,среднее,1,гражданский брак,1,F,пенсионер,0,2194,на проведение свадьбы
8583,0,2194,58,высшее,0,не женат / не замужем,4,F,пенсионер,0,2194,дополнительное образование
9238,2,2194,34,среднее,1,женат / замужем,0,F,сотрудник,0,2194,покупка жилья для сдачи
9528,0,2194,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,2194,операции со своей недвижимостью
9627,0,2194,56,среднее,1,женат / замужем,0,F,пенсионер,0,2194,операции со своей недвижимостью
10462,0,2194,62,среднее,1,женат / замужем,0,F,пенсионер,0,2194,покупка коммерческой недвижимости


In [24]:
df = df.drop_duplicates().reset_index(drop = True) #удаляет дубликаты, правит индексы
df.duplicated().sum() #подсчитывает количество дупликатов после операции

0

Теперь разберемся с регистром данных в столбце `education`

In [25]:
df.education = df.education.str.lower() #приводим значения к нижнему регистру
df.education.value_counts()

среднее                15188
высшее                  5251
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64

**Вывод**

Мы удалили полные дупликаты из таблицы и изменили регистр данных в столбце `education`. Полные дубликаты искажают данные таблицы, делают ее тяжелее. В столбце `education` мы теперь имеем корректные данные для анализа.

### Лемматизация

Столбец purpose содержит в себе неявные дубликаты - одинаковые цели кредита сформулированы по-разному. Для приведения столбца к читаемому виду, нужно лемматизировать строки и понять, каковы основные цели кредитов.

In [26]:
m = Mystem()
purposes = df.purpose.to_list()      #Приводим столбец purpose в лист
df.purpose.value_counts()
purposes_str = ' '.join(purposes)    #Приводим лист в текст
lemmas = m.lemmatize(purposes_str)   #Лемматизируем текст и подсчитываем количество слов
Counter(lemmas)

Counter({'покупка': 5899,
         ' ': 55064,
         'жилье': 4461,
         'приобретение': 461,
         'автомобиль': 4308,
         'дополнительный': 907,
         'образование': 4014,
         'сыграть': 769,
         'свадьба': 2335,
         'операция': 2604,
         'с': 2918,
         'на': 2228,
         'проведение': 773,
         'для': 1290,
         'семья': 638,
         'недвижимость': 6352,
         'коммерческий': 1312,
         'жилой': 1231,
         'строительство': 1879,
         'собственный': 635,
         'подержать': 853,
         'свой': 2231,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'получение': 1315,
         'высокий': 1374,
         'подержанный': 111,
         'профильный': 436,
         'сдача': 652,
         'ремонт': 607,
         '\n': 1})

Как мы можем видеть, основных целей кредита четыре. Это "автомобиль", "образование", "свадьба", "недвижимость". Заменим строки в `purpose` на эти цели.

In [27]:
def purpose_category(purposes):   #Пишем функцию, которая заменяет значение в столбце на категориальные 
    if 'свадьб' in purposes:
        return 'свадьба'
    if 'жиль' in purposes or 'недвижим' in purposes:
        return 'недвижимость'
    if 'автомоб' in purposes:
        return 'автомобиль'
    if 'образов' in purposes:
        return 'образование'
    return 'иные цели'

df.purpose = df.purpose.apply(purpose_category) #Применяем функцию к столбцу purpose
df.purpose.value_counts()

недвижимость    10813
автомобиль       4308
образование      4014
свадьба          2335
Name: purpose, dtype: int64

**Вывод**

С помощью лемматизации были распознаны дубликаты в столбце `purpose`, что позволило обработать их и привести к читаемому виду. Уже сейчас видно, что чаще всего клиенты берут кредит на недвижимость, а количество кредитов на личное средство передвижения и на образование примерно равны.

### Категоризация данных

Для ответа на поставленные вопросы нужно привести таблицу в удобный для визуализации и работы вид - выделить отдельные интересующие нас столбцы с данными и обработать.

In [28]:
children_debt = df[['children', 'debt']] #создаем новый словарь со столбцами children и debt
children_debt.head()

,children,debt
0,1,0
1,1,0
2,0,0
3,3,0
4,0,0


In [29]:
family_status_debt = df[['family_status', 'debt']]
family_status_debt.head()

,family_status,debt
0,женат / замужем,0
1,женат / замужем,0
2,женат / замужем,0
3,женат / замужем,0
4,гражданский брак,0


Проведем категоризацию данных о доходе по категориям "ниже среднего", "средний", "выше среднего". Для этого напишем функцию и применим ее к каждому значению столбца total_income. Сохраним результат в новом столбце income_category

In [30]:
income_median = df.total_income.median() #сохраним медианную зарплату в переменной

def income_categorizer(income): 
    if income < income_median:
        return 'ниже среднего'
    if income == income_median:
        return 'средний'
    return 'выше среднего'

df['income_category'] = df['total_income'].apply(income_categorizer)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,выше среднего
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,ниже среднего
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,выше среднего
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,выше среднего
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,выше среднего


Теперь объединим столбцы `debt`, `total_income` и `income_category` в словарь

In [31]:
income_debt = df[['total_income', 'income_category', 'debt']]
income_debt.head()

,total_income,income_category,debt
0,253875,выше среднего,0
1,112080,ниже среднего,0
2,145885,выше среднего,0
3,267628,выше среднего,0
4,158616,выше среднего,0


In [32]:
purpose_debt = df[['purpose', 'debt']]
purpose_debt.head()

,purpose,debt
0,недвижимость,0
1,автомобиль,0
2,недвижимость,0
3,образование,0
4,свадьба,0


**Вывод**

Мы объединили столбцы таблицы `df` по интересующим нас данным. В столбце `total_income` мы провели категоризацию по медианном доходу для удобства. Данные приведены в порядок, выделены необходимые словари, мы готовы к анализу и ответам на вопросы.

## Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Используя словари, созданные на прошлом этапе работы, проверим, клиенты с каким количеством детей чаще имеют долги по кредитам

In [33]:
children_values = children_debt.children.value_counts() 
children_values

0    14106
1     4856
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

In [34]:
debt_values = children_debt[children_debt.debt == 1]['children'].value_counts() 
debt_values
#Считаем количество клиентов с долгами, сортируем по количеству детей

0    1063
1     445
2     202
3      27
4       4
Name: children, dtype: int64

По цифрам похоже, что люди без детей гораздо чаще не платят по кредиту, нежели люди с большим количеством детей. Однако, посчитаем отношение должников к общему количеству клиентов.

In [35]:
debt_values / children_values 

0    0.075358
1    0.091639
2    0.094925
3    0.081818
4    0.097561
5         NaN
Name: children, dtype: float64

Только 7,5% клиентов без детей являются должниками, в то время как среди многодетных семей (4 и больше детей) эта цифра достигает 9,8%!

**Вывод**

Согласно данным, многодетные семьи в среднем чаще не платят по кредитам. Однако, разница невелика - всего 2.3%.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [36]:
family_count = family_status_debt.family_status.value_counts()
family_count

женат / замужем          12344
гражданский брак          4162
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [37]:
family_debt_count = family_status_debt[family_status_debt.debt == 1]['family_status'].value_counts()
family_debt_count

женат / замужем          931
гражданский брак         388
не женат / не замужем    274
в разводе                 85
вдовец / вдова            63
Name: family_status, dtype: int64

In [38]:
family_debt_count / family_count

женат / замужем          0.075421
гражданский брак         0.093224
не женат / не замужем    0.097509
в разводе                0.071130
вдовец / вдова           0.065693
Name: family_status, dtype: float64

**Вывод**

Данные показывают, что процент должников выше среди клиентов, живущих в гражданском браке и не живущих в браке вообще. В то же время среди людей, потерявших супруга, самый низкий процент долгов по кредиту.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [39]:
income_count = income_debt.income_category.value_counts()
income_count

выше среднего    10735
ниже среднего    10735
Name: income_category, dtype: int64

In [40]:
income_debt_count = income_debt[income_debt.debt == 1]['income_category'].value_counts()
income_debt_count

ниже среднего    874
выше среднего    867
Name: income_category, dtype: int64

In [41]:
income_debt_count / income_count

выше среднего    0.080764
ниже среднего    0.081416
Name: income_category, dtype: float64

**Вывод**

По результатам анализа можно видеть, что уровень дохода практически не влияет на возврат кредита в срок. Стоит упомянуть, что это достаточно грубый анализ, так как деление уровня дохода на выше и ниже среднего не самый точный.

- Как разные цели кредита влияют на его возврат в срок?

In [42]:
purpose_count = purpose_debt.purpose.value_counts()
purpose_count

недвижимость    10813
автомобиль       4308
образование      4014
свадьба          2335
Name: purpose, dtype: int64

In [43]:
purpose_debt_count = purpose_debt[purpose_debt.debt == 1]['purpose'].value_counts()
purpose_debt_count

недвижимость    782
автомобиль      403
образование     370
свадьба         186
Name: purpose, dtype: int64

In [44]:
purpose_debt_count / purpose_count

недвижимость    0.072320
автомобиль      0.093547
образование     0.092177
свадьба         0.079657
Name: purpose, dtype: float64

**Вывод**

Чаще всего не платят по кредиту клиенты, взявшие его на покупку автомобиля или на оплату образования. 

## Общий вывод

В ходе работы были выполнены следующие цели:
1) Были обнаружены и заменены пустые значения в столбцах дохода и стажа  
2) Данные в столбце стажа были приведены к положительным значениям, кроме того изменен тип данных стажа и дохода с `float` на `int`  
3) Данные в столбце `education` были приведены к единому регистру  
4) В столбце `purpose` используя лемматизацию были найдены основные цели кредитов, дупликаты были вычищены и заменены основными целями  
5) Для ответа на поставленные вопросы были созданы отдельные словари  
6) Используя словари, было найдено отношение должников к общему количеству клиентов, разбитым на категории "наличие детей", "семейное положение", "уровень дохода", "цели кредита". На основании полученных данных были сделаны выводы и даны ответы на поставленные вопросы